In [8]:
"""
air_inflow: 공기 흡입 유량 (^3/min)
air_end_temp: 공기 말단 온도 (°C)
out_pressure: 토출 압력 (Mpa)
motor_current: 모터 전류 (A)
motor_rpm: 모터 회전수 (rpm)
motor_temp: 모터 온도 (°C)
motor_vibe: 모터 진동 (mm/s)
type: 설비 번호

설비 번호 [0, 4, 5, 6, 7]: 30HP(마력)
설비 번호 1: 20HP
설비 번호 2: 10HP
설비 번호 3: 50HP
"""

'\nair_inflow: 공기 흡입 유량 (^3/min)\nair_end_temp: 공기 말단 온도 (°C)\nout_pressure: 토출 압력 (Mpa)\nmotor_current: 모터 전류 (A)\nmotor_rpm: 모터 회전수 (rpm)\nmotor_temp: 모터 온도 (°C)\nmotor_vibe: 모터 진동 (mm/s)\ntype: 설비 번호\n\n설비 번호 [0, 4, 5, 6, 7]: 30HP(마력)\n설비 번호 1: 20HP\n설비 번호 2: 10HP\n설비 번호 3: 50HP\n'

In [53]:
import pandas as pd
from sklearn.model_selection import train_test_split

from src.features import build_features

In [65]:
df = pd.read_csv(r'C:\Project_MB\Anomaly-Detection-of-Air-Compressor\data\raw\train_data.csv')
test = pd.read_csv(r'C:\Project_MB\Anomaly-Detection-of-Air-Compressor\data\raw\test_data.csv')

In [66]:
df = build_features.add_motor_hp(df)
df = build_features.add_motor_hp(df)
df = build_features.add_motor_vibe_freq(df)

test = build_features.add_motor_hp(test)
test = build_features.add_motor_hp(test)
test = build_features.add_motor_vibe_freq(test)

In [67]:
from sklearn.ensemble import IsolationForest

# 모델 학습 및 이상치 탐지
model = IsolationForest(n_estimators=100, max_samples='auto', contamination=0.05, random_state=42)
model.fit(df)

c:\Users\kki96\anaconda3\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but IsolationForest was fitted with feature names
  warnings.warn(


IsolationForest(contamination=0.05, random_state=42)

In [68]:
test_score = test.copy()
test_score["anomaly_score"] = model.decision_function(test_score)

In [75]:
test_score["anomaly"] = test_score["anomaly_score"].apply(lambda x: 0 if x > 0 else 1)


In [76]:
test_score.anomaly.value_counts()

0    7112
1     277
Name: anomaly, dtype: int64

In [77]:
test_score

,air_inflow,air_end_temp,out_pressure,motor_current,motor_rpm,motor_temp,motor_vibe,type,motor_hp,motor_vibe_freq1,motor_vibe_freq2,anomaly_score,anomaly
0,2.51,53.28,0.7,32.54,2662.0,69.58,3.48,0,30,-424.423282,940.44368,0.120374,0
1,2.66,55.24,0.7,34.45,2819.0,71.32,3.57,0,30,-424.423282,940.44368,0.114207,0
2,1.72,42.74,0.7,22.23,1819.0,60.21,3.01,0,30,-424.423282,940.44368,0.137700,0
3,2.20,49.15,0.7,28.50,2332.0,65.91,3.30,0,30,-424.423282,940.44368,0.129234,0
4,2.06,47.28,0.7,26.67,2182.0,64.24,3.21,0,30,-424.423282,940.44368,0.142592,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
7384,2.12,48.08,0.7,27.45,2246.0,64.96,3.25,7,30,-424.423282,940.44368,0.108989,0
7385,1.48,39.63,0.7,19.19,1570.0,57.44,2.87,7,30,-424.423282,940.44368,0.097162,0
7386,1.56,40.61,0.7,20.15,1649.0,58.32,2.92,7,30,-424.423282,940.44368,0.108996,0
7387,1.59,40.99,0.7,20.52,1679.0,58.66,2.93,7,30,-424.423282,940.44368,0.106299,0


In [79]:
submission = pd.read_csv(r'C:\Project_MB\Anomaly-Detection-of-Air-Compressor\data\raw\answer_sample.csv')
submission

,type,label
0,0,-1
1,0,-1
2,0,-1
3,0,-1
4,0,-1
...,...,...
7384,7,-1
7385,7,-1
7386,7,-1
7387,7,-1


In [80]:
submission["label"] = test_score["anomaly"]

In [83]:
submission.to_csv(r'C:\Project_MB\Anomaly-Detection-of-Air-Compressor\data\submission\submission.csv', index=False)